In [59]:
# first need to construct the data

import os
import sys
parent = os.path.realpath(os.path.join(os.path.abspath(''), os.pardir))
sys.path.append(parent)

import numpy as np
from databases.sql_connect import create_connection
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from copy import deepcopy
import plotly.graph_objects as go
import plotly.express as pex
import holoviews as hv
from holoviews import dim
from bokeh.io import export_png
import floweaver
from floweaver import *
#from ipysankeywidget import SankeyWidget

category = "GPR-Minerals- Total"
category = "CED - total"
category = "EDIP-Gold"
category = "GPR-Minerals- Total"
category = "ADP-Minerals- Total"
category = "7122:Gold"
category = "CML-climate change (GWP 100a)"
category = "GPR-Minerals- Total"
category = "CML-climate change (GWP 100a)"

conn = None
sns.set_theme()
sns.set(context="paper", style="darkgrid", font="Times New Roman")
cm = 1/2.54
DB_CORR = os.path.join(parent,"databases", "TEMPRO_DB230416_Corr.db")
assert(os.path.exists(DB_CORR))
FONTSIZE = 8


conn = create_connection(DB_CORR)

cats ={
    "CML-climate change (GWP 100a)": "Green House Emissions",
    "CED - total": "Primary Energy",
    "ADP-Minerals- Total": "Mineral Depletion", 
    "EI-Minerals- Total": "Economic Importance",
    "GPR-Minerals- Total":"Geo-Politic Supply Risk", 
    "EDIP-Gold": "Gold -Resource",
    "7122:Gold": "Gold Content",
    "7103:Copper": "Copper Content",
    "EDIP-Copper": "Copper -Resource",
    "7168:Tantalum": "Tantalum Content",
    "ReCiPe Endpoint (E - A) - total (total)": "Total Impacts",
}

ps = 6001


In [60]:
def get_lca_contributions(ps_list, category, group=6000):
    query = f'SELECT * FROM [{group}LCAResults]'
    lca = pd.read_sql_query(query, conn)
    lca = lca[lca["ProductSystemID"].isin(ps_list)]
    lca = lca[lca["Category"] == category]
    lca.set_index("ProductSystemID", inplace=True)
    #lca.drop(columns=["ID","StandardDev","DataQuality"], inplace=True)
    return lca

def get_inventory(group):
    queryInv = f'SELECT * FROM [{group}Exchanges]'
    inv = pd.read_sql_query(queryInv, conn)
    inv["Contribution"] = np.nan
    return inv


In [61]:

#for category in cats.keys():
#if True:
#category="CED - total"# "EDIP-Gold"#"""CED - total"  #"GPR-Minerals- Total" # "ADP-Minerals- Total"#"CED - total" 
def get_sankey_data_category(category):
    df_subsubcats = pd.read_excel(
        "2000subsubcats.xlsx", engine="openpyxl", sheet_name="Subsubcat")
    df_subsubcats.set_index("ID", inplace=True)


    dfs = []

    # Need Data of the Inventory as well
    lca6 = get_lca_contributions([ps],category, group=6000)
    inv6 = get_inventory(group=6000)
    inv6 = inv6[inv6["6000ID"] == ps]

    # then need the data of the level below
    list5 = list(set(inv6[inv6["6000ID"] == ps]["5000Systems"]))
    lca5 = get_lca_contributions(list5, category, group=5000)

    # Now multiply the amount * impact to get cnotrigution
    for idx in inv6.index:
        inv6.at[idx, "Subsubcat"] = "5_" + \
            df_subsubcats.at[inv6.at[idx, "5000Systems"], "Subsubcat Short"]
        inv6.at[idx, "Contribution"] = inv6.at[idx, "Amount"] * \
            lca5.at[inv6.at[idx, "5000Systems"], "Result"]

    inv6["Subsubcat_to"] = np.nan
    for idx in inv6.index:
        inv6.at[idx, "Subsubcat_to"] = "6_" + \
            df_subsubcats.at[inv6.at[idx, "6000ID"], "Subsubcat Short"]

    sankey6 = {"source": inv6["Subsubcat"],
                "target": inv6["Subsubcat_to"],
                "value": inv6["Contribution"]}
    sankey_data6 = pd.DataFrame.from_dict(sankey6)
    
    # inv6.to_csv("inv6.csv")


    # get the information from 4000 to 5000
    lca5 = get_lca_contributions(list5,category, group=5000)
    inv5 = get_inventory(group=5000)
    inv5 = inv5[inv5["5000ID"].isin(list5)]
    inv5["Subsubcat"] = np.nan

    # then need the data of the level below
    list4 = list(set(inv5[inv5["5000ID"].isin(list5)]["4000Devices"]))
    # there are also from 300s and 2000s, more on that later
    list4 = [x for x in list4 if x is not None]
    lca4 = get_lca_contributions(list4, category, group=4000)

    # Now multiply the amount * impact to get cnotrigution
    for idx in inv5.index:
        try:
            ref_amount = df_subsubcats.at[inv5.at[idx,
                                                    "4000Devices"], "Ref Unit"]
            inv5.at[idx, "Contribution"] = inv5.at[idx, "Amount"] * \
                lca4.at[inv5.at[idx, "4000Devices"], "Result"]/ref_amount
            inv5.at[idx, "Subsubcat"] = "4_" + \
                df_subsubcats.at[inv5.at[idx, "4000Devices"], "Subsubcat Short"]
        except:
            pass

    # then need the data of the level below
    list3 = list(set(inv5[inv5["5000ID"].isin(list5)]["3000Pieces"]))
    # there are also from 300s and 2000s, more on that later
    list3 = [x for x in list3 if x is not None]
    lca3 = get_lca_contributions(list3, category, group=3000)

    # Now multiply the amount * impact to get cnotrigution
    for idx in inv5.index:
        try:
            inv5.at[idx, "Contribution"] = inv5.at[idx, "Amount"] * \
                lca3.at[inv5.at[idx, "3000Pieces"], "Result"]
            inv5.at[idx, "Subsubcat"] = "3_" + \
                df_subsubcats.at[inv5.at[idx, "3000Pieces"], "Subsubcat Short"]
        except:
            pass

    list2 = list(set(inv5[inv5["5000ID"].isin(list5)]["2000Parts"]))
    # there are also from 300s and 2000s, more on that later
    list2 = [x for x in list2 if x is not None]
    lca2 = get_lca_contributions(list2, category, group=2000)

    # Now multiply the amount * impact to get cnotrigution
    for idx in inv5.index:
        try:
            inv5.at[idx, "Contribution"] = inv5.at[idx, "Amount"] * \
                lca2.at[inv5.at[idx, "2000Parts"], "Result"]
            inv5.at[idx, "Subsubcat"] = "2_" + \
                df_subsubcats.at[inv5.at[idx, "2000Parts"], "Subsubcat Short"]
        except:
            pass
    inv5["Subsubcat_to"] = np.nan

    for idx in inv5.index:
        inv5.at[idx, "Subsubcat_to"] = "5_" + \
            df_subsubcats.at[inv5.at[idx, "5000ID"], "Subsubcat Short"]

    sankey5 = {"source": inv5["Subsubcat"],
                "target": inv5["Subsubcat_to"],
                "value": inv5["Contribution"]}

    sankey_data5 = pd.DataFrame.from_dict(sankey5)
    sankey_data5 = sankey_data5.groupby(['source', 'target']).sum().reset_index()


    # get sankey data from 3 AND 2 to 4
    lca4 = get_lca_contributions(list4, category, group=4000)
    inv4 = get_inventory(group=4000)
    inv4 = inv4[inv4["4000ID"].isin(list4)]
    inv4["Subsubcat"] = np.nan
    inv4["Subsubcat_out"] = np.nan
    inv4["Amount_3000"] = np.nan
    inv4["Amount_5000"] = np.nan
    inv4["ref_amount"] = np.nan

    # then need the data of the level below
    list3 = list(set(inv4[inv4["4000ID"].isin(list4)]["3000Pieces"]))
    # there are also from 300s and 2000s, more on that later
    list3 = [x for x in list3 if x is not None]
    lca3 = get_lca_contributions(list3,category, group=3000)

    # Now multiply the amount * impact to get cnotrigution
    for idx in inv4.index:
        demand_5000 = inv5[inv5["4000Devices"] ==
                            inv4.at[idx, "4000ID"]]["Amount"].sum()
        inv4.at[idx, "Amount_5000"] = demand_5000
        ref_amount = df_subsubcats.at[inv4.at[idx, "4000ID"], "Ref Unit"]
        inv4.at[idx, "ref_amount"] = ref_amount
        amount = inv4.at[idx, "Amount"]
        try:
            inv4.at[idx, "Contribution"] = inv4.at[idx, "Amount"] * \
                lca3.at[inv4.at[idx, "3000Pieces"],
                        "Result"]/ref_amount*demand_5000
            inv4.at[idx, "Amount_3000"] = inv4.at[idx,
                                                    "Amount"]/ref_amount*demand_5000
            inv4.at[idx, "Subsubcat"] = "3_" + \
                df_subsubcats.at[inv4.at[idx, "3000Pieces"], "Subsubcat Short"]
        except:
            pass
    # Level 2
    list2 = list(set(inv4[inv4["4000ID"].isin(list4)]["2000Parts"]))
    # there are also from 300s and 2000s, more on that later
    list2 = [x for x in list2 if x is not None]
    lca2 = get_lca_contributions(list2,category, group=2000)

    # Now multiply the amount * impact to get cnotrigution
    for idx in inv4.index:
        demand_5000 = inv5[inv5["4000Devices"] ==
                            inv4.at[idx, "4000ID"]]["Amount"].sum()
        ref_amount = df_subsubcats.at[inv4.at[idx, "4000ID"], "Ref Unit"]
        inv4.at[idx, "ref_amount"] = ref_amount
        inv4.at[idx, "Amount_5000"] = demand_5000
        try:
            inv4.at[idx, "Contribution"] = inv4.at[idx, "Amount"] * \
                lca2.at[inv4.at[idx, "2000Parts"],
                        "Result"]/ref_amount*demand_5000
            name = df_subsubcats.at[inv4.at[idx, "2000Parts"], "Subsubcat Short"]

            if not isinstance(name, str):
                inv4.at[idx, "Subsubcat"] = "2_Other"
            else:
                inv4.at[idx, "Subsubcat"] = "2_" + \
                    df_subsubcats.at[inv4.at[idx,
                                                "2000Parts"], "Subsubcat Short"]
        except:
            pass
    #######################################
    # there are contributions on the same level, make with care
    ####################
    # Level 4
    list4_a = list(set(inv4[inv4["4000ID"].isin(list4)]["4000Devices"]))
    list4_a = [x for x in list4_a if x is not None]
    lca4_a = get_lca_contributions(list4_a,category, group=4000)

    # Now multiply the amount * impact to get cnotrigution
    query = f'SELECT * FROM [4000Devices]'
    list_ps_4000 = pd.read_sql_query(query, conn)
    list_ps_4000.set_index("ID", inplace=True)

    for idx in inv4.index:
        demand_5000 = inv5[inv5["4000Devices"] ==
                            inv4.at[idx, "4000ID"]]["Amount"].sum()
        ref_amount = df_subsubcats.at[inv4.at[idx, "4000ID"], "Ref Unit"]
        inv4.at[idx, "Amount_5000"] = demand_5000
        inv4.at[idx, "ref_amount"] = ref_amount

        try:
            # dividir para la ref amount de cada 4000
            ref_amount_out = list_ps_4000.at[inv4.at[idx,
                                                        "4000Devices"], "Amount"]
            inv4.at[idx, "Contribution"] = inv4.at[idx, "Amount"] * lca4_a.at[inv4.at[idx,
                                                                                        "4000Devices"], "Result"]/ref_amount*demand_5000/ref_amount_out
            inv4.at[idx, "Subsubcat"] = "4_" + \
                df_subsubcats.at[inv4.at[idx, "4000Devices"], "Subsubcat Short"]
        except:
            pass

    for idx in inv4.index:
        inv4.at[idx, "Subsubcat_out"] = "4_" + \
            df_subsubcats.at[inv4.at[idx, "4000ID"], "Subsubcat Short"]

    inv4 = inv4[inv4["Contribution"].notna()]

    inv4_a = deepcopy(inv4)
    # get those with 4446
    inv4_a_small_it = inv4_a[inv4_a["4000Devices"] == 4446]
    table_small_it = inv4_a[inv4_a["4000ID"] == 4446]
    ref_amount = 18
    idx_4000_to_delete = []
    dfs_add = []
    for idx in inv4_a_small_it.index:

        total_contrib = inv4_a_small_it.at[idx, "Contribution"]
        amount = inv4_a_small_it.at[idx, "Amount"]
        amount_5000 = inv4_a_small_it.at[idx, "Amount_5000"]
        ref_amount__4000 = inv4_a_small_it.at[idx, "ref_amount"]
        scaling = amount*amount_5000/ref_amount__4000/ref_amount
        df_samall_it_new = deepcopy(table_small_it)
        idx_4000_to_delete.append(idx)
        df_samall_it_new["Amount"] *= scaling
        df_samall_it_new["Contribution"] *= scaling
        df_samall_it_new["Amount_3000"] *= scaling
        # inv4_a_small_it.at[idx, "Amount_5000"]/inv4_a_small_it.at[idx, "ref_amount"]
        df_samall_it_new["Amount_5000"] = 1
        df_samall_it_new["Subsubcat_out"] = inv4_a_small_it.at[idx,
                                                                "Subsubcat_out"]
        new_contrib = df_samall_it_new["Contribution"].sum()
        dfs_add.append(df_samall_it_new)

    inv4_a.drop(idx_4000_to_delete, axis=0, inplace=True)
    inv4_b = pd.concat([inv4_a, *dfs_add], ignore_index=True)
    inv4_b = inv4_b[inv4_b["4000Devices"] != 4446]
    inv4_b.to_csv("inv4_b.csv")

    sankey4 = {"source": inv4_b["Subsubcat"],
                "target": inv4_b["Subsubcat_out"],
                "value": inv4_b["Contribution"]}
    sankey_data4 = pd.DataFrame.from_dict(sankey4)
    sankey_data4 = sankey_data4.groupby(['source', 'target']).sum().reset_index()

    #inv4_b.to_csv("inv4_b.csv")
    inv4_reqs3000 = deepcopy(inv4_b)
    inv4_reqs3000 = inv4_reqs3000[inv4_reqs3000["Amount_3000"].notna()]
    inv4_reqs3000 = inv4_reqs3000[["3000Pieces", "Amount_3000"]]
    inv4_reqs3000 = inv4_reqs3000.groupby('3000Pieces').sum()

    lca3 = get_lca_contributions(list3, category,group=3000)
    inv3 = get_inventory(group=3000)
    inv3 = inv3[inv3["3000ID"].isin(list3)]
    inv3["Contribution"] = np.nan
    inv3["Subsubcat"] = np.nan
    inv3["Subsubcat_out"] = np.nan
    inv3["2000_demand"] = np.nan

    # then need the data of the level below
    list2 = list(set(inv3[inv3["3000ID"].isin(list3)]["2000Parts"]))
    list2 = [x for x in list2 if x is not None]
    list2 = [x for x in list2 if x != ""]

    # there are also from 300s and 2000s, more on that later

    lca2 = get_lca_contributions(list2, category,group=2000)
    # Now multiply the amount * impact to get cnotrigution
    for idx in inv3.index:
        id_3000 = inv3.at[idx, "3000ID"]
        demand_3000 = inv4_reqs3000.at[id_3000, "Amount_3000"]
        piece_id_exchange = inv3.at[idx, "2000Parts"]
        if not isinstance(piece_id_exchange, str):
            if piece_id_exchange in lca2.index:
                inv3.at[idx, "Contribution"] = inv3.at[idx, "Amount"] * \
                    lca2.at[piece_id_exchange, "Result"]*demand_3000
                inv3.at[idx, "2000_demand"] = inv3.at[idx, "Amount"]*demand_3000
                name = df_subsubcats.at[piece_id_exchange, "Subsubcat Short"]

                if not isinstance(name, str):
                    inv3.at[idx, "Subsubcat"] = "2_Other"
                else:
                    inv3.at[idx, "Subsubcat"] = "2_" + \
                        df_subsubcats.at[inv3.at[idx,
                                                    "2000Parts"], "Subsubcat Short"]

    # then need the data of the level below
    list3_a = list(set(inv3[inv3["3000ID"].isin(list3)]["3000Pieces"]))
    list3_a = [x for x in list3_a if x is not None]
    lca3_a = get_lca_contributions(list3_a,category, group=3000)

    # Now multiply the amount * impact to get cnotrigution
    for idx in inv3.index:
        id_3000 = inv3.at[idx, "3000ID"]
        demand_3000 = inv4_reqs3000.at[id_3000, "Amount_3000"]

        piece_id_exchange = inv3.at[idx, "3000Pieces"]
        if not isinstance(piece_id_exchange, str):
            if piece_id_exchange in lca3_a.index:
                inv3.at[idx, "Contribution"] = inv3.at[idx, "Amount"] * \
                    lca3_a.at[piece_id_exchange, "Result"]*demand_3000
                inv3.at[idx, "Subsubcat"] = "3_" + \
                    df_subsubcats.at[piece_id_exchange, "Subsubcat Short"]
    for idx in inv3.index:
        inv3.at[idx, "Subsubcat_out"] = "3_" + \
            df_subsubcats.at[inv3.at[idx, "3000ID"], "Subsubcat Short"]

        if inv3.at[idx, "Subsubcat_out"] == inv3.at[idx, "Subsubcat"]:
            # inv3.at[idx,"Subsubcat"]="2_ffff"+inv3.at[idx,"Subsubcat"]
            name = inv3.at[idx, "Subsubcat"]
            inv3.at[idx, "Subsubcat"] = "2_Other"  # f"3_3_{name}/{name}"

        if inv3.at[idx, "Subsubcat"] == "3_PCB" and inv3.at[idx, "Subsubcat_out"] == "3_Small IT":
            print("found")
            inv3.at[idx, "Subsubcat"] = "2_PWB"

    sankey3 = {"source": inv3["Subsubcat"],
                "target": inv3["Subsubcat_out"],
                "value": inv3["Contribution"]}
    sankey_data3 = pd.DataFrame.from_dict(sankey3)
    sankey_data3 = sankey_data3[sankey_data3["source"].notna()]

    sankey_data = pd.concat(
        [sankey_data6, sankey_data5, sankey_data4,sankey_data3], ignore_index=True)


    for idx in sankey_data.index:
        sankey_data.at[idx, "source"] = sankey_data.at[idx,
                                                     "source"].replace("4_Energy Supply", "4_Energy Supply Dev.")
        sankey_data.at[idx, "target"] = sankey_data.at[idx,
                                                   "target"].replace("4_Energy Supply", "4_Energy Supply Dev.")
        sankey_data.at[idx, "source"] = sankey_data.at[idx,
                                                     "source"].replace("4_Infrastructure", "4_Infrastructure Dev")
        sankey_data.at[idx, "target"] = sankey_data.at[idx,
                                                   "target"].replace("4_Infrastructure", "4_Infrastructure Dev")
        sankey_data.at[idx, "source"] = sankey_data.at[idx,
                                                     "source"].replace("4_Climatization", "4_Clima. Dev")
        sankey_data.at[idx, "target"] = sankey_data.at[idx,
                                                   "target"].replace("4_Climatization", "4_Clima. Dev")

    sankey_data = sankey_data.groupby(['source', 'target']).sum().reset_index()

    return sankey_data




In [62]:
sankey_data_og = get_sankey_data_category("CML-climate change (GWP 100a)")


In [70]:
import copy
sankey_data= copy.deepcopy(sankey_data_og)
total_impact = 0
for idx in sankey_data.index:
    if sankey_data.at[idx,"target"][0:2]=="6_":
        total_impact+=sankey_data.at[idx,"value"]

sankey_data["value"] *= (1/total_impact)*100
sankey_data = sankey_data[sankey_data["value"] > 1]
unique_vas_source = [x for x in sankey_data["source"].unique()]
unique_vas_tar = [x for x in sankey_data["target"].unique()]
unique_vars = list(set([*unique_vas_source, *unique_vas_tar]))
unique_6 = [x[2:] for x in unique_vars if "6" in x]
unique_5 = [x[2:] for x in unique_vars if "5" in x]
unique_4 = [x[2:] for x in unique_vars if "4" in x]
unique_3 = [x[2:] for x in unique_vars if "3" in x]
unique_2 = [x[2:] for x in unique_vars if "2" in x]
unique_3 = ["PCB"] + [x for x in unique_3 if x !="PCB"]
unique_2 = ["IC", "PWB"] + [x for x in unique_2 if x not in ["IC", "PWB"]]

unique_5.reverse()

for idx in sankey_data.index:
    for i in [2, 3, 4, 5, 6]:
        sankey_data.at[idx, "source"] = sankey_data.at[idx,
                                                        "source"].replace(f"{i}_", "")
        sankey_data.at[idx, "target"] = sankey_data.at[idx,
                                                    "target"].replace(f"{i}_", "")

nodes = {
    'Pieces': ProcessGroup(unique_2, title='Pieces'),
    'Parts': ProcessGroup(unique_3, title='Parts'),
    'Devices': ProcessGroup(unique_4, title='Devices'),
    'Systems': ProcessGroup(unique_5, title='Systems'),
    'DC': ProcessGroup(unique_6, title='Data Center'),
}

ordering = [
    ['Pieces'],
    ['Parts'],
    ['Devices'],
    ['Systems'],
    ['DC']        
]

bundles = [Bundle('Pieces', 'Devices'),
           Bundle('Pieces', 'Parts'),
           Bundle('Pieces', 'Systems'),
           Bundle('Parts', 'Devices'),
           Bundle('Parts', 'Systems'),
           Bundle('Devices', 'Systems'),
           Bundle('Systems', 'DC'),]
           
if True:
    partition_2 = Partition.Simple("process", unique_2)
    partition_3 = Partition.Simple("process", unique_3)
    partition_4 = Partition.Simple("process", unique_4)
    partition_5 = Partition.Simple("process", unique_5)
    partition_6 = Partition.Simple("process", unique_6)
    nodes["Pieces"].partition = partition_2
    nodes["Parts"].partition = partition_3
    nodes["Devices"].partition = partition_4
    nodes["Systems"].partition = partition_5
    nodes["DC"].partition = partition_6

    sdd = SankeyDefinition(nodes, bundles, ordering)

    class MyScale(QuantitativeScale):
        def get_palette(self, link):
            # Choose colour scheme based on link type (here, Employment Job)
            name = "YlGnBu_4" #'PuBuGn_9'
            return self.lookup_palette_name(name)

        def get_color(self, link, value):
            palette = self.get_palette(link)
            return palette(0.2+ 0.8*value)

    scale = MyScale('value', palette='Oranges_9')
    f=40
    w = int(25)*f
    h = int(10)*f

weave(sdd, sankey_data,link_color=scale).to_widget(width=w, height=h, margins=dict(left=120, right=100))



SankeyWidget(groups=[{'id': 'Pieces', 'type': 'process', 'title': 'Pieces', 'nodes': ['Pieces^IC', 'Pieces^PWB…

In [64]:
sankey_data_rec=deepcopy(sankey_data)

# the amount of "recovered material" is a product of the pcbs* gold content
query = f'SELECT * FROM [6000LCAResults] WHERE ProductSystemID={ps}'
lca = pd.read_sql_query(query, conn)

lca["AssoID"]=np.nan
for idx in lca.index:
    if lca.at[idx,"Category"].count(":")==2:
        lca.at[idx,"AssoID"]=int(lca.at[idx,"Category"].split(":")[1])
lca=lca[lca["AssoID"]>3000]
lca=lca[["AssoID", "Result"]]
list_3000 = list(lca["AssoID"])
lca.set_index("AssoID", inplace=True)

lca3000s = get_lca_contributions(list_3000,category="7122:Gold",group=3000)
#lca.drop(columns=["ID","StandardDev","DataQuality"], inplace=True)
lca3000s 

lca["UnitaryGoldContent"]=np.nan
lca["TotalGoldContent"]=np.nan
for idx in lca.index:
    lca.at[idx,"UnitaryGoldContent"]=lca3000s.at[idx,"Result"]
    lca.at[idx,"TotalGoldContent"]=lca.at[idx,"Result"]*lca3000s.at[idx,"Result"]
total_PCB_GOLD = lca["TotalGoldContent"].sum()





In [65]:
sankey_data = get_sankey_data_category(category="7122:Gold")#"EDIP-Gold"))
# Here I would prefer that the total "NATURE INPUT" is the total EDIP AMOUNT
GOLD_FACTOR = 1.09006
CRUSH_RATE = 0.7
"source	target	value"
#sankey_data["value"] *= (1/total_impact)*100
total_impact = 0

for idx in sankey_data.index:
    if sankey_data.at[idx,"target"][0:2]=="6_":
        total_impact+=sankey_data.at[idx,"value"]
        id_6 = sankey_data.at[idx,"target"]

#make_new_flows 
## eol flows
#A portion from the output is not in PCBs and not recovered
not_in_pcb = total_impact-total_PCB_GOLD

# 30 is lost to crushing
crushing_waste = (1-CRUSH_RATE)*total_PCB_GOLD
crushing_content =CRUSH_RATE*total_PCB_GOLD

# pyrolysis has a 98% efficiency
pyro_waste = 0.02*crushing_content
pyro_recover= 0.98*crushing_content
flow_waste_EOL = [id_6, "Non Recovered", not_in_pcb]
flow_output_EOL = [id_6, "EOL Treatment", total_PCB_GOLD]

flow_crushing_waste= ["EOL Treatment", "Non Recovered", crushing_waste]
flow_crushing_content= ["EOL Treatment", "Crushed_PCB", crushing_content]
flow_pyro_waste = ["Crushed_PCB", "Non Recovered", pyro_waste]
flow_pyro_recover = ["Crushed_PCB", "Recovered", pyro_recover]
for flow in [flow_waste_EOL, flow_output_EOL,flow_crushing_waste, flow_crushing_content,flow_pyro_waste,flow_pyro_recover]:
    sankey_data.loc[len(sankey_data.index)]=flow

flows_gold_2 = 0
for idx in sankey_data.index:
    if sankey_data.at[idx,"source"][0:2]=="2_":
        flows_gold_2+=sankey_data.at[idx,"value"]

total_gold_demand = flows_gold_2*1.02
reduction_factor = 1-pyro_recover/total_gold_demand

# Input from nature:
for idx in sankey_data.index:
    if sankey_data.at[idx,"source"][0:2]=="2_":
        
        new_flow = ["1_Gold", sankey_data.at[idx,"source"], sankey_data.at[idx,"value"]] 
        new_flow_metal_waste = ["1_Gold", "Waste", sankey_data.at[idx,"value"]*0.02] 
        new_flow_metal_prod = ["0_Env", "1_Gold", sankey_data.at[idx,"value"]*1.02*reduction_factor] 
        new_flow_metal_prod_waste = ["0_Env", "Waste", sankey_data.at[idx,"value"]*1.02*(GOLD_FACTOR-1)*reduction_factor] 
        sankey_data.loc[len(sankey_data.index)]=new_flow
        sankey_data.loc[len(sankey_data.index)]=new_flow_metal_waste
        sankey_data.loc[len(sankey_data.index)]=new_flow_metal_prod
        sankey_data.loc[len(sankey_data.index)]=new_flow_metal_prod_waste



unique_eol=["EOL Treatment"]
unique_crush=["Crushed_PCB"]
unique_pyro=["Recovered"]
unique_non_recov=["Non Recovered"]

sankey_data = sankey_data.groupby(['source', 'target']).sum().reset_index()

sankey_data.loc[len(sankey_data.index)]=["From Recovery", "1_Gold",pyro_recover]
sankey_data.to_csv("Sankey_Recycling.csv")
print([total_impact, flows_gold_2,total_PCB_GOLD, pyro_recover])



[0.06746129116521746, 0.0673945445824077, 0.06328694283698635, 0.043414842786172636]


In [66]:

#print(total_impact)
from copy import deepcopy
sankey_data_a=deepcopy(sankey_data)
sankey_data_a["value"] *= (1/total_impact)*100
sankey_data_a = sankey_data_a[sankey_data_a["value"] > 1]

unique_vas_source = [x for x in sankey_data_a["source"].unique()]
unique_vas_tar = [x for x in sankey_data_a["target"].unique()]
unique_vars = list(set([*unique_vas_source, *unique_vas_tar]))

unique_6 = [x[2:] for x in unique_vars if "6" in x]
unique_5 = [x[2:] for x in unique_vars if "5" in x]
unique_4 = [x[2:] for x in unique_vars if "4" in x]
unique_3 = [x[2:] for x in unique_vars if "3" in x]
unique_2 = [x[2:] for x in unique_vars if "2" in x]
unique_1 = [x[2:] for x in unique_vars if "1" in x] # Gold Metal
unique_0 = [x[2:] for x in unique_vars if "0" in x] # Environment
unique_waste = ["Waste"] # Waste
unique_eol=["EOL Treatment"]
unique_crush=["Crushed_PCB"]
unique_pyro=["Recovered"]
unique_non_recov=["Non Recovered"]
unique_from_recov=["From Recovery"]

unique_5.reverse()
print(unique_waste)

['Waste']


In [67]:

for idx in sankey_data_a.index:
    for i in range(0, 9):
        sankey_data_a.at[idx, "source"] = sankey_data_a.at[idx,
                                                           "source"].replace(f"{i}_", "")
        sankey_data_a.at[idx, "target"] = sankey_data_a.at[idx,
                                                           "target"].replace(f"{i}_", "")

nodes = {
    "Env": ProcessGroup(unique_0, title='Env.'),
    "FromR": ProcessGroup(unique_from_recov, title='Recov.'),
    "Metal": ProcessGroup(unique_1, title='Metal'),
    "Waste": ProcessGroup(unique_waste, title='Waste'),
    'Pieces': ProcessGroup(unique_2, title='Pieces'),
    'Parts': ProcessGroup(unique_3, title='Parts'),
    'Devices': ProcessGroup(unique_4, title='Devices'),
    'Systems': ProcessGroup(unique_5, title='Systems'),
    'DC': ProcessGroup(unique_6, title='Data Center'),
    "EOL": ProcessGroup(unique_eol, title='EOL'),
    "Treatment": ProcessGroup(unique_crush, title='Treatment'),
    "Recycling": ProcessGroup(unique_pyro, title='Recovered'),
    "NonRec": ProcessGroup(unique_non_recov, title='NonRecov'),

}

ordering = [
    [[], ["Env", "FromR"]],
    [[], ["Metal"]],
    [["Waste"], ['Pieces']],
    [[], ['Parts']],
    [[], ['Devices']],
    [[], ['Systems']],
    [[], ['DC']],
    [[], ['EOL']],
    [[], ['Treatment']],
    [["NonRec"], ['Recycling']],

]

bundles = [
    Bundle('Env', 'Metal'),
    Bundle('FromR', 'Metal'),

    Bundle('Env', 'Waste'),
    Bundle('Metal', 'Pieces'),
    Bundle('Metal', 'Waste'),

    Bundle('Pieces', 'Devices'),
    Bundle('Pieces', 'Parts'),
    Bundle('Pieces', 'Systems'),
    Bundle('Parts', 'Devices'),
    Bundle('Parts', 'Systems'),
    Bundle('Devices', 'Systems'),
    Bundle('Systems', 'DC'),
    Bundle('DC', 'EOL'),
    Bundle('EOL', 'Treatment'),
    Bundle('Treatment', "Recycling"),
    Bundle('DC', 'NonRec'),
    Bundle('EOL', 'NonRec'),
    Bundle('Treatment', "NonRec"),]

#sankey_data["value"] *= (1/total_impact)*100

partition_2 = Partition.Simple("process", unique_2)
partition_3 = Partition.Simple("process", unique_3)
partition_4 = Partition.Simple("process", unique_4)
partition_5 = Partition.Simple("process", unique_5)
partition_6 = Partition.Simple("process", unique_6)
nodes["Pieces"].partition = partition_2
nodes["Parts"].partition = partition_3
nodes["Devices"].partition = partition_4
nodes["Systems"].partition = partition_5
nodes["DC"].partition = partition_6

sdd = SankeyDefinition(nodes, bundles, ordering)


class MyScale(QuantitativeScale):
    def get_palette(self, link):
        # Choose colour scheme based on link type (here, Employment Job)
        name = 'YlGnBu_4'
        return self.lookup_palette_name(name)

    def get_color(self, link, value):
        palette = self.get_palette(link)
        return palette(0.1 + 0.9*value)


scale = MyScale('value', palette='YlGnBu_5')
f = 40
w = int(42)*f
h = int(14)*f

weave(sdd, sankey_data_a, link_color=scale).to_widget(
    width=w, height=h, margins=dict(left=60, right=100))


SankeyWidget(groups=[{'id': '__Env_Waste_1', 'type': 'group', 'title': '', 'nodes': ['__Env_Waste_1^*']}, {'id…

In [68]:

import pandas as pd
from io import StringIO

flows = pd.read_csv(StringIO("""
source,target,type,value
a,b,main,1.5
a,c,main,1
b,d,main,2
c,d,main,1
d,e,main,0.5
d,o, main,2.5
e,b,back,0.5
"""))


In [69]:

from floweaver import *

# Set the default size to fit the documentation better.
size = dict(width=570, height=300)

nodes = {
    'a': ProcessGroup(['a']),
    'b': ProcessGroup(['b']),
    'c': ProcessGroup(['c']),
    'd': ProcessGroup(['d']),
    'e': ProcessGroup(['e']),
    'o': ProcessGroup(['o']),
    'back': Waypoint(direction='L'),
}

bundles = [
    Bundle('e', 'b'),
    Bundle('a', 'b'),
    Bundle('a', 'c'),
    Bundle('b', 'd'),
    Bundle('c', 'd'),
    Bundle('d', 'e'),
    Bundle('d', 'o'),


]

ordering = [
    [['a'],[]],
    [[ 'c',"b"],["back"]],
    [ ['d'],[]],
        [["o"],["e"]],
]

sdd = SankeyDefinition(nodes, bundles, ordering)

weave(sdd, flows).to_widget(**size)

SankeyWidget(groups=[{'id': '__e_b_1', 'type': 'group', 'title': '', 'nodes': ['__e_b_1^*']}, {'id': '__e_b_2'…